In [1]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
from sklearn.cluster import OPTICS, cluster_optics_dbscan

/home/ivo/Programming_Personal_Projects/cluster_proj/clustering-problem/.ct_env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L3-v2')


Downloading: 100%|██████████| 190/190 [00:00<00:00, 127kB/s]
Downloading: 100%|██████████| 4.01k/4.01k [00:00<00:00, 3.52MB/s]
Downloading: 100%|██████████| 122/122 [00:00<00:00, 144kB/s]
Downloading: 100%|██████████| 69.6M/69.6M [00:06<00:00, 10.3MB/s]
Downloading: 100%|██████████| 53.0/53.0 [00:00<00:00, 32.6kB/s]
Downloading: 100%|██████████| 112/112 [00:00<00:00, 73.3kB/s]
Downloading: 100%|██████████| 466k/466k [00:01<00:00, 363kB/s]  
Downloading: 100%|██████████| 314/314 [00:00<00:00, 145kB/s]
Downloading: 100%|██████████| 232k/232k [00:02<00:00, 94.2kB/s]
Downloading: 100%|██████████| 229/229 [00:00<00:00, 200kB/s]


In [4]:
data = pd.read_csv("../data/location_data.csv",index_col=None)

In [5]:
data.describe()

,city,road,house_number,postcode,state
count,63695,63695,63695,63695,63695
unique,5534,17184,6353,11015,121
top,houston,main st,101,46227,ca
freq,640,599,357,33,3251


In [6]:
data.head()

,city,road,house_number,postcode,state
0,fort worth,sycamore school rd,5421,76123,texas
1,gautier,highway 90,2701,39553,mississippi
2,farmington,e main,4750,87401,nm
3,richland,queensgate drive,2751,99352,washington
4,davie,s state hwy seven,1600,33317,fl


In [7]:
data.shape

(63695, 5)

In [8]:
data_no_dup = data.drop_duplicates(ignore_index=True) 
# we don't want to keep the old indexes
data_no_dup.shape # we have many duplicate rows

(41423, 5)

In [10]:
data_no_dup

,city,road,house_number,postcode,state
0,fort worth,sycamore school rd,5421,76123,texas
1,gautier,highway 90,2701,39553,mississippi
2,farmington,e main,4750,87401,nm
3,richland,queensgate drive,2751,99352,washington
4,davie,s state hwy seven,1600,33317,fl
...,...,...,...,...,...
41418,highland,baseline st,27774,92346,california
41419,hilliard,main st,3685,43026,ohio
41420,window rock,window rock loop road,264,86515,az
41421,denver,s santa fe dr,1395,80223,co


In [11]:
sentences = data_no_dup["city"]+" "+ data_no_dup["road"]+" "+ data_no_dup["house_number"]\
            +" "+ data_no_dup["postcode"]+" "+ data_no_dup["state"]

In [12]:
sentences[34357] # this doesn't seem to be an address in the USA...

'kota jkt utara jl. boulevard artha gading jalan kav no.01 rt.18/rw.8 klp. gading bar. kec. klp. gading 18 14240 daerah khusus ibukota jakarta'

In [13]:
sentence_embeddings = model.encode(sentences)
# this will not scale well...

In [14]:
pd.DataFrame(sentence_embeddings).to_csv("../data/embeddings.csv")

In [15]:
clustering = OPTICS(cluster_method = "dbscan",
                    n_jobs = -1).fit(sentence_embeddings.astype("float16"))

In [25]:
labels_results = clustering.labels_

In [26]:
pd.DataFrame(labels_results).to_csv("../data/labels_results.csv")